# Authentication and Authorization

Django provides a robust system for Authentication (verifying the identity of a user) and Authorization (determining what a user can do). This system helps secure web applications and is tightly integrated with Django's models and views.



## 1. Understanding Authentication and Authorization

Authentication: Confirming the identity of a user (e.g., login/logout functionality).

Authorization: Managing permissions and determining if a user can perform specific actions.

### Django includes:

User Model: A default user model to manage users.

Authentication Views: Login, logout, and password management.

Permissions: Fine-grained control over user actions.

Groups: Logical grouping of permissions for easier management

# 2. Setting Up Authentication

Basic User Model

Django’s built-in User model is located in django.contrib.auth.models.

Default fields in User:

Username

Password

Email

First Name

Last Name

Permissions and groups

#### Registering Users

Create a user registration system.

#### Create a Registration Form

Define a custom form for user registration:

In [ ]:
# forms.py
from django import forms
from django.contrib.auth.models import User
from django.contrib.auth.forms import UserCreationForm

class RegistrationForm(UserCreationForm):
    email = forms.EmailField(required=True)

    class Meta:
        model = User
        fields = ['username', 'email', 'password1', 'password2']


### Handle Registration in Views

Use the form in a view to handle registration:

In [ ]:
# views.py
from django.shortcuts import render, redirect
from .forms import RegistrationForm

def register(request):
    if request.method == 'POST':
        form = RegistrationForm(request.POST)
        if form.is_valid():
            form.save()
            return redirect('login')  # Redirect to the login page
    else:
        form = RegistrationForm()

    return render(request, 'register.html', {'form': form})


### Template for Registration
Render the registration form in a template:

In [ ]:
<!-- register.html -->
<!DOCTYPE html>
<html>
<head>
    <title>Register</title>
</head>
<body>
    <h2>Register</h2>
    <form method="post">
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit">Register</button>
    </form>
</body>
</html>


### Login and Logout

Django's Built-in Authentication Views
### Django provides built-in views for login and logout:

LoginView: Handles user login.

LogoutView: Handles user logout.

Configure these views in urls.py:

In [ ]:
# urls.py
from django.urls import path
from django.contrib.auth.views import LoginView, LogoutView

urlpatterns = [
    path('login/', LoginView.as_view(template_name='login.html'), name='login'),
    path('logout/', LogoutView.as_view(next_page='login'), name='logout'),
]


### Template for Login
Create a login form:

In [ ]:
<!-- login.html -->
<!DOCTYPE html>
<html>
<head>
    <title>Login</title>
</head>
<body>
    <h2>Login</h2>
    <form method="post">
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit">Login</button>
    </form>
</body>
</html>


### Redirect After Login
Set the default redirect location:

In [ ]:
# settings.py
LOGIN_REDIRECT_URL = '/'  # Redirect after login
LOGOUT_REDIRECT_URL = '/login/'  # Redirect after logout


### 3. Managing Permissions
Permissions determine what actions a user can perform.


Assigning Permissions

Permissions are automatically created for models (add, change, delete, view).

Use Django Admin to assign permissions to users/groups.

Checking Permissions in Views

Use the @permission_required decorator:

In [ ]:
from django.contrib.auth.decorators import permission_required

@permission_required('app_name.permission_code')
def restricted_view(request):
    return HttpResponse("You have permission!")


### 4. User Groups

Groups are collections of permissions that you can assign to multiple users.

Creating and Assigning Groups

a) Create Groups via Admin Panel


Navigate to Django Admin.
Create a group and assign permissions.


b) Assign Groups to Users
Assign users to groups via the Admin panel or programmatically:

In [ ]:
from django.contrib.auth.models import Group, User

group = Group.objects.get(name='Editors')
user = User.objects.get(username='john_doe')
user.groups.add(group)


Check Group Membership

In [ ]:
if request.user.groups.filter(name='Editors').exists():
    print("User is in the Editors group!")


### 5. Using the LoginRequiredMixin
Restrict views to authenticated users:

In [ ]:
from django.contrib.auth.mixins import LoginRequiredMixin
from django.views.generic import TemplateView

class DashboardView(LoginRequiredMixin, TemplateView):
    template_name = 'dashboard.html'


For function-based views:

In [ ]:
from django.contrib.auth.decorators import login_required

@login_required
def dashboard(request):
    return render(request, 'dashboard.html')


### 6. Password Management
Django provides built-in views for password management.

Password Change

Add the PasswordChangeView to urls.py:

In [ ]:
from django.contrib.auth.views import PasswordChangeView

urlpatterns = [
    path('password_change/', PasswordChangeView.as_view(template_name='password_change.html'), name='password_change'),
]


### Password Reset
Django provides a full password reset workflow. Add these views to urls.py:

In [ ]:
from django.contrib.auth import views as auth_views

urlpatterns += [
    path('password_reset/', auth_views.PasswordResetView.as_view(), name='password_reset'),
    path('password_reset_done/', auth_views.PasswordResetDoneView.as_view(), name='password_reset_done'),
    path('reset/<uidb64>/<token>/', auth_views.PasswordResetConfirmView.as_view(), name='password_reset_confirm'),
    path('reset_done/', auth_views.PasswordResetCompleteView.as_view(), name='password_reset_complete'),
]


### 7. Customizing the User Model
If the default User model doesn't meet your needs, you can create a custom user model.

Define a Custom User Model

In [ ]:
from django.contrib.auth.models import AbstractUser
from django.db import models

class CustomUser(AbstractUser):
    phone_number = models.CharField(max_length=15, blank=True, null=True)


Update Settings

In [ ]:
# settings.py
AUTH_USER_MODEL = 'your_app.CustomUser'


### Create Forms for the Custom User

In [ ]:
from django.contrib.auth.forms import UserCreationForm
from .models import CustomUser

class CustomUserCreationForm(UserCreationForm):
    class Meta:
        model = CustomUser
        fields = ['username', 'email', 'phone_number']
